In [40]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as co
import timeit

In [83]:
class State:
    def __init__(self, T, rho, m_a, eps, sig, r_cut, N_a):
        self.k_BT = T*co.k
        self.eps = eps*co.k
        self.sig2 = (sig*1e-10)**2
        self.r_cut2 = (r_cut*1e-10)**2
        self.initial_config(N_a, rho, m_a)
    
    def initial_config(self, N_a, rho, m_a):
        self.L = np.power(m_a*N_a/(co.N_A*1000*rho), 1/3)
        self.x = np.random.rand(3, N_a)*self.L - self.L/2
    
    def single_energy(self, N_i, trial=False):
        if trial==False:
            x = self.x
        else:
            x = self.x_trial

        # Calculate all atomic distances
        rel_r = (x - x[:, N_i].reshape(3, 1) + self.L/2) % self.L - self.L/2
        d2 = np.einsum('ij, ij -> j', rel_r, rel_r)
        # Check for self-interaction and cutoff
        d2[N_i] = 2*self.r_cut2
        d2 = d2[d2 < self.r_cut2]
        
        # Calculate powers of sig/r
        sr2 = self.sig2 / d2
        sr6 = sr2*sr2*sr2
        sr12 = sr6*sr6
        
        # Sum up every energy interaction
        return np.sum(4*self.eps*(sr12 - sr6))
    
    def total_energy(self):
        e_tot = 0
        for i in range(np.shape(self.x)[1]):
            rel_r = (self.x[:, 0:i] - self.x[:, i].reshape(3, 1) + self.L/2) % self.L - self.L/2
            d2 = np.einsum('ij, ij -> j', rel_r, rel_r)

            # take out the cutoff
            d2 = d2[d2 < self.r_cut2]
            
            # Calculate powers of sig/r
            sr2 = self.sig2 / d2
            sr6 = sr2*sr2*sr2
            sr12 = sr6*sr6
            
            # Sum up every energy interaction
            e_tot += np.sum(4*self.eps*(sr12 - sr6))
        return e_tot

    def total_energy2(self):
        N, D = self.x.shape
        r = np.broadcast_to(self.x, (D, N, N))


In [88]:
Simulation = State(150, 361.4, 16.04246, 148, 3.73, 14, 10)
e_singe = Simulation.single_energy(0)
e_tot = Simulation.total_energy()
e_tot2 = Simulation.total_energy2()

In [133]:
a = np.random.rand(3, 4)
print(a)
aa = np.broadcast_to(a, (4, 3, 4))
print(np.shape(aa))

[[0.20587526 0.65292532 0.30686544 0.07503841]
 [0.72699383 0.74289657 0.91480996 0.62028321]
 [0.29464287 0.251744   0.42578062 0.70112574]]
(4, 3, 4)


In [78]:
%%timeit
e = Simulation.single_energy(0)

140 µs ± 3.66 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [79]:
%%timeit
e = Simulation.total_energy()

92.3 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [87]:
%%timeit
e = Simulation.total_energy2()

171 ms ± 7.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
